## Query and edit Database

In [ ]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv("MONGODBURI")
client = MongoClient(uri, server_api=ServerApi("1"))
try:
    client.admin.command("ping")
    print("Success")
except Exception as e:
    print(e)

db = client["test"]["Leads"]
cursor = db.find({}, {"_id": 0})
df = pd.DataFrame(list(cursor))[["name", "headline", "location", "link"]]
df

Success


,name,headline,location,link
0,Fahim ul Haq,Co-Founder & CEO at Educative | Software Engineer,"Bellevue, Washington, United States",https://www.linkedin.com/in/fahimulhaq
1,Yash Patel,2nd Year Engineering Student @ Downing College...,United Kingdom,https://www.linkedin.com/in/yash-patel-a34113226
2,Md Bulbul Hossain,I'm a Professional Digital Marketing & White H...,"Sirajganj District, Rajshahi, Bangladesh",https://www.linkedin.com/in/mdbulbulhossain
3,Lars-Thorsten Sudmann,"🔝 Reduce Costs, Increase Revenue, Use the Powe...","Gevelsberg, North Rhine-Westphalia, Germany",https://www.linkedin.com/in/ltsudmann
4,Ayesha Mansha,Co-CEO @ Brand ClickX | I help with Elite SEO ...,"Albany, New York, United States",https://www.linkedin.com/in/ayesha-mansha-seo-...
5,Adyan Shahid,"Student at Queen Elizabeth's School, Barnet","Stevenage, England, United Kingdom",https://www.linkedin.com/in/adyan-shahid-ba621...
6,Neil Enright,"Headmaster at Queen Elizabeth's School, Barnet","Barnet, England, United Kingdom",https://www.linkedin.com/in/neil-enright-ba32a...
7,Rayan Nadeem,"Student at Queen Elizabeth's School, Barnet | ...","Pinner, England, United Kingdom",https://www.linkedin.com/in/rayan-nadeem-13861...


### Class

In [37]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

class Database:
    """
    Connection to MongoDB Database and load to Pandas
    """
    def __init__(self): # initialise connection to DB
        load_dotenv()
        uri = os.getenv("MONGODBURI")
        self.uri = uri
        self.df = None

    def getDb(self): # Retrieve information from database
        client = MongoClient(self.uri)
        db = client["test"]["Leads"]
        cursor = db.find({})
        df = pd.DataFrame(list(cursor))
        self.df = df
        client.close()

    def update(self, filter: dict, replacement: dict): # Update record
        """
        filter: {"field": "value"}
        replacement: {"name": name, "link": link}
        """
        try:
            client = MongoClient(self.uri)
            collection = client["test"]["Leads"]
            result = collection.replace_one(filter, replacement)
            self.getDb()
            client.close()
            return result.modified_count > 0
        except Exception as e:
            raise Exception("Unable to edit document due to the following error: ", e)

In [41]:
db = Database()
db.getDb()
db.df

,_id,link,headline,location,name
0,68a118137b2f5850278a152a,https://www.linkedin.com/in/fahimulhaq,Co-Founder & CEO at Educative | Software Engineer,"Bellevue, Washington, United States",Fahim ul Haq
1,68a118137b2f5850278a152b,https://www.linkedin.com/in/yash-patel-a34113226,2nd Year Engineering Student @ Downing College...,United Kingdom,Yash Patel
2,68a118ae345a16303fd65786,https://www.linkedin.com/in/mdbulbulhossain,I'm a Professional Digital Marketing & White H...,"Sirajganj District, Rajshahi, Bangladesh",Md Bulbul Hossain
3,68a118ae345a16303fd65787,https://www.linkedin.com/in/ltsudmann,"🔝 Reduce Costs, Increase Revenue, Use the Powe...","Gevelsberg, North Rhine-Westphalia, Germany",Lars-Thorsten Sudmann
4,68a118ae345a16303fd65788,https://www.linkedin.com/in/ayesha-mansha-seo-...,Co-CEO @ Brand ClickX | I help with Elite SEO ...,"Albany, New York, United States",Ayesha Mansha
5,68a1cc560a68109981293f05,https://www.linkedin.com/in/adyan-shahid-ba621...,"Student at Queen Elizabeth's School, Barnet","Stevenage, England, United Kingdom",Adyan Shahid
6,68a1cc560a68109981293f06,https://www.linkedin.com/in/neil-enright-ba32a...,"Headmaster at Queen Elizabeth's School, Barnet","Barnet, England, United Kingdom",Neil Enright
7,68a1cc560a68109981293f07,https://www.linkedin.com/in/rayan-nadeem-13861...,"Student at Queen Elizabeth's School, Barnet | ...","Pinner, England, United Kingdom",Rayan Nadeem


# Playground

### How to use .update()

In [40]:
db.update({"name": "Jeeve Singh"}, {
    "name": "JS", 
    "location": "London, England, UK",
    "headline": "Student at UCL",
    "new parameter": "test"
    }) # Returns false since Zain N not found in DB
db.df

,_id,name,location,headline,new parameter,link
0,68a118137b2f5850278a1529,JS,"London, England, UK",Student at UCL,test,NaN
1,68a118137b2f5850278a152a,Fahim ul Haq,"Bellevue, Washington, United States",Co-Founder & CEO at Educative | Software Engineer,NaN,https://www.linkedin.com/in/fahimulhaq
2,68a118137b2f5850278a152b,Yash Patel,United Kingdom,2nd Year Engineering Student @ Downing College...,NaN,https://www.linkedin.com/in/yash-patel-a34113226
3,68a118ae345a16303fd65786,Md Bulbul Hossain,"Sirajganj District, Rajshahi, Bangladesh",I'm a Professional Digital Marketing & White H...,NaN,https://www.linkedin.com/in/mdbulbulhossain
4,68a118ae345a16303fd65787,Lars-Thorsten Sudmann,"Gevelsberg, North Rhine-Westphalia, Germany","🔝 Reduce Costs, Increase Revenue, Use the Powe...",NaN,https://www.linkedin.com/in/ltsudmann
5,68a118ae345a16303fd65788,Ayesha Mansha,"Albany, New York, United States",Co-CEO @ Brand ClickX | I help with Elite SEO ...,NaN,https://www.linkedin.com/in/ayesha-mansha-seo-...
6,68a1cc560a68109981293f05,Adyan Shahid,"Stevenage, England, United Kingdom","Student at Queen Elizabeth's School, Barnet",NaN,https://www.linkedin.com/in/adyan-shahid-ba621...
7,68a1cc560a68109981293f06,Neil Enright,"Barnet, England, United Kingdom","Headmaster at Queen Elizabeth's School, Barnet",NaN,https://www.linkedin.com/in/neil-enright-ba32a...
8,68a1cc560a68109981293f07,Rayan Nadeem,"Pinner, England, United Kingdom","Student at Queen Elizabeth's School, Barnet | ...",NaN,https://www.linkedin.com/in/rayan-nadeem-13861...
